In [1]:
## Instalaciones

%pip install torch
%pip install open3d
%pip install tabulate

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
## Dependencias

from typing import List
import torch
import os
import open3d as o3d
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from model import PointNetClassifier, PointNetLoss, PointNetKAN, TTAClassifier
from modelnet10 import ModelNetClass, ModelNet, DatasetType
from utils.csv import save_loss_dict
from utils.transformation import (Normalization,
                                  Rotation, Translation, Reflection, Scale,
                                  DropRandom, DropSphere, Jittering, Noise)
from trainer import PointNetTrainer


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {DEVICE}.")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Using cuda.


In [3]:
# parámetros globales
checkpoint_freq = 25

# parámetros del dataset
classes = [label for label in ModelNetClass]
batch_size = 32
dim = 3
num_points = 1024
num_classes = len(classes)

# hiperparámetros
num_global_feats = 1024     # número de features globales calculadas
learning_rate = 0.001
reg_weight = 0.001
gamma = 2                   # Recomendado por el paper de focal loss

# TODO: Más adelante usar alpha para clases imbalanceadas

In [4]:

# dataset de entrenamiento
t = [Rotation(), Reflection(), Scale(max_ratio=2.5),
    Jittering(max_units=0.005), DropRandom(loss_ratio=0.4), Noise()]

train_data = ModelNet(classes, DatasetType.TRAIN, repetitions=3, transformations=t, preserve_original=False)
validation_data = ModelNet(classes, DatasetType.VALIDATION, repetitions=3, transformations=t, preserve_original=False)

In [5]:
# Función de entrenamiento
def train(
        epochs: int,
        name: str,
        num_global_feats: int,
        learning_rate: int,
        use_scheduler: bool,
        alpha: List[int],
        gamma: int,
        reg_weight: int,
        use_kan: bool,
        ignore_Tnet: bool,
        norm_type: str,
        dropout: float
):
    if not use_kan:
        classifier = PointNetClassifier(dim, num_points, num_global_feats, num_classes, ignore_Tnet=ignore_Tnet,
                                        norm_type=norm_type, dropout=dropout).to(DEVICE)
    else:
        classifier = PointNetKAN(dim, num_points, num_classes, scaling = 2.0).to(DEVICE)
    optimizer = optim.Adam(classifier.parameters(), lr=learning_rate)
    if DEVICE == "cuda" and use_scheduler:
        scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.01, step_size_up=2000, cycle_momentum=False)
    else:
        scheduler = None
    
    trainer = PointNetTrainer(
        name=name,
        model=classifier,
        optimizer=optimizer,
        scheduler=scheduler,
        criterion=PointNetLoss(alpha=alpha, gamma=gamma, reg_weight=reg_weight, size_average=True).to(DEVICE),
        device=DEVICE,
        train_loader=DataLoader(train_data, batch_size=batch_size, shuffle=True),
        val_loader=DataLoader(validation_data, batch_size=batch_size, shuffle=False),
        checkpoint_dir=os.path.join(os.getcwd(), "checkpoint"),
        checkpoint_freq=checkpoint_freq
    )

    loss_dict, best_epoch, best_loss, best_acc = trainer.fit(epochs=epochs)
    #save_loss_dict(loss_dict, os.path.join(os.getcwd(), "csv", f"{name}_loss_dict.csv"))
    print(f"{name} | Best model @ epoch {best_epoch}: loss = {best_loss:.4f}, acc = {best_acc:.4f}")

# Instancias de entrenamiento
EPOCHS=200

In [6]:
train(epochs=EPOCHS, name="gn", num_global_feats=num_global_feats, learning_rate=learning_rate,
      use_scheduler=False, alpha=None, gamma=0, reg_weight=reg_weight, use_kan=False, ignore_Tnet=False,
      norm_type="groupnorm", dropout=0.3)

╒═══════════╤══════════════╤═════════════╤════════════╤═══════════╕
│ Epoch     │   Train Loss │   Train Acc │   Val Loss │   Val Acc │
╞═══════════╪══════════════╪═════════════╪════════════╪═══════════╡
│ Epoch 1   │       2.1567 │      0.2948 │     1.9611 │    0.3121 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 2   │       2.0165 │      0.3419 │     1.8607 │    0.3521 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 3   │       1.9302 │      0.3502 │     1.8371 │    0.3562 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 4   │       1.8638 │      0.363  │     1.7381 │    0.4012 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 5   │       1.7789 │      0.3869 │     1.7114 │    0.3862 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 6   │       1.7244 │      0.4019 │     1.6737 │    0.4109 │
├───────────┼──────────────┼─────────────┼──────

In [7]:
train(epochs=EPOCHS, name="lessdp", num_global_feats=num_global_feats, learning_rate=learning_rate,
      use_scheduler=False, alpha=None, gamma=0, reg_weight=reg_weight, use_kan=False, ignore_Tnet=False,
      norm_type="batchnorm", dropout=0.1)

╒═══════════╤══════════════╤═════════════╤══════════════════╤═══════════╕
│ Epoch     │   Train Loss │   Train Acc │         Val Loss │   Val Acc │
╞═══════════╪══════════════╪═════════════╪══════════════════╪═══════════╡
│ Epoch 1   │       2.2273 │      0.292  │      1.986       │    0.3454 │
├───────────┼──────────────┼─────────────┼──────────────────┼───────────┤
│ Epoch 2   │       2.0142 │      0.3557 │      1.827       │    0.3811 │
├───────────┼──────────────┼─────────────┼──────────────────┼───────────┤
│ Epoch 3   │       1.9197 │      0.381  │      1.7823      │    0.4081 │
├───────────┼──────────────┼─────────────┼──────────────────┼───────────┤
│ Epoch 4   │       1.8273 │      0.4133 │      1.6827      │    0.4258 │
├───────────┼──────────────┼─────────────┼──────────────────┼───────────┤
│ Epoch 5   │       1.7675 │      0.4274 │      1.6704      │    0.4421 │
├───────────┼──────────────┼─────────────┼──────────────────┼───────────┤
│ Epoch 6   │       1.6955 │      0.44

In [8]:
train(epochs=EPOCHS, name="moredp", num_global_feats=num_global_feats, learning_rate=learning_rate,
      use_scheduler=False, alpha=None, gamma=0, reg_weight=reg_weight, use_kan=False, ignore_Tnet=False,
      norm_type="batchnorm", dropout=0.5)

╒═══════════╤══════════════╤═════════════╤══════════════════╤═══════════╕
│ Epoch     │   Train Loss │   Train Acc │         Val Loss │   Val Acc │
╞═══════════╪══════════════╪═════════════╪══════════════════╪═══════════╡
│ Epoch 1   │       2.3421 │      0.2614 │      2.0689      │    0.2996 │
├───────────┼──────────────┼─────────────┼──────────────────┼───────────┤
│ Epoch 2   │       2.1322 │      0.3026 │      1.9221      │    0.3046 │
├───────────┼──────────────┼─────────────┼──────────────────┼───────────┤
│ Epoch 3   │       2.1047 │      0.3192 │      1.9554      │    0.3042 │
├───────────┼──────────────┼─────────────┼──────────────────┼───────────┤
│ Epoch 4   │       2.0133 │      0.3428 │      1.8727      │    0.383  │
├───────────┼──────────────┼─────────────┼──────────────────┼───────────┤
│ Epoch 5   │       1.8912 │      0.3809 │      1.7595      │    0.4096 │
├───────────┼──────────────┼─────────────┼──────────────────┼───────────┤
│ Epoch 6   │       1.8253 │      0.40

In [11]:
train(epochs=EPOCHS, name="gnreg", num_global_feats=num_global_feats, learning_rate=learning_rate,
      use_scheduler=False, alpha=None, gamma=2, reg_weight=0.003, use_kan=False, ignore_Tnet=False,
      norm_type="groupnorm", dropout=0.4)

╒═══════════╤══════════════╤═════════════╤════════════╤═══════════╕
│ Epoch     │   Train Loss │   Train Acc │   Val Loss │   Val Acc │
╞═══════════╪══════════════╪═════════════╪════════════╪═══════════╡
│ Epoch 1   │       1.7668 │      0.2774 │     1.492  │    0.3262 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 2   │       1.5783 │      0.3353 │     1.4933 │    0.3212 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 3   │       1.5189 │      0.3484 │     1.3793 │    0.3733 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 4   │       1.4592 │      0.3602 │     1.3661 │    0.3475 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 5   │       1.402  │      0.3687 │     1.2714 │    0.3959 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 6   │       1.3282 │      0.3897 │     1.2283 │    0.4041 │
├───────────┼──────────────┼─────────────┼──────

# Dataset Testeo

In [9]:
# dataset de prueba
base_test_data = ModelNet(classes, DatasetType.TEST, repetitions=1, preserve_original=False,
                          transformations=[])
affine_test_data = ModelNet(classes, DatasetType.TEST, repetitions=1, preserve_original=False,
                          transformations=[Rotation(), Reflection(), Scale(max_ratio=2.5)])
complex_test_data = ModelNet(classes, DatasetType.TEST, repetitions=1, preserve_original=False,
                          transformations=[Rotation(), Reflection(), Scale(max_ratio=2.5),
                                          Jittering(max_units=0.005), DropRandom(loss_ratio=0.4), Noise()])

In [15]:
def test_it(classifier_path: str, num_global_feats=num_global_feats, use_kan=False,
            ignore_Tnet=False, use_TTA=False, merge_mode=None, norm_type="batchnorm"):

    for data_name, data in [["base", base_test_data], ["affine", affine_test_data], ["complex", complex_test_data]]:
        data_loader = DataLoader(data, batch_size=batch_size, shuffle=False)
    
        if not use_kan:
            classifier = PointNetClassifier(dim, num_points, num_global_feats, num_classes,
                                            ignore_Tnet=ignore_Tnet, norm_type=norm_type).to(DEVICE)
        else:
            classifier = PointNetKAN(dim, num_points, num_classes, scaling = 1.0, ignore_Tnet=ignore_Tnet).to(DEVICE)
        
        
        classifier.load_state_dict(torch.load(classifier_path, map_location=torch.device(DEVICE)))

        if use_TTA:
            classifier = TTAClassifier(classifier=classifier, transformations=[Rotation(), Reflection()], merge_mode=merge_mode)

        
            
        with torch.no_grad():
            classifier = classifier.eval()
            correct = 0
            
            for pcds, labels in data_loader:
                pcds = pcds.to(DEVICE)
                labels = labels.squeeze().to(DEVICE)

                # Hacer predicciones
                out, _, _ = classifier(pcds)

                if not use_TTA:
                    # Calculamos las elecciones, TTA ya hace softmax.
                    out = torch.softmax(out, dim=1)
                    
                pred_choice = out.argmax(dim=1)
                
                # Elecciones correctas, acumuladas
                correct += pred_choice.eq(labels.data).cpu().sum().item()

            test_acc = correct / float(len(data))
            print(f"\tAccuracy on {data_name} dataset:\t", test_acc)

# Tests
_dir = os.path.join(os.getcwd(), "checkpoint", "best_model")
print("Groupnorm classifier (best epoch):")
test_it(os.path.join(_dir, "gn_best_model.pth"), norm_type="groupnorm")
print("Less dropout classifier (best epoch):")
test_it(os.path.join(_dir, "lessdp_best_model.pth"))
print("More dropout classifier (best epoch):")
test_it(os.path.join(_dir, "moredp_best_model.pth"))
print("Groupnorm with regularization classifier (best epoch):")
test_it(os.path.join(_dir, "gnreg_best_model.pth"), norm_type="groupnorm")

print("Groupnorm classifier (last epoch):")
test_it(os.path.join(os.getcwd(), "checkpoint", "by_epoch", "gn_epoch_0200.pth"), norm_type="groupnorm")
print("Less dropout classifier (last epoch):")
test_it(os.path.join(os.getcwd(), "checkpoint", "by_epoch", "lessdp_epoch_0200.pth"))
print("More dropout classifier (last epoch):")
test_it(os.path.join(os.getcwd(), "checkpoint", "by_epoch", "moredp_epoch_0200.pth"))
print("Groupnorm with regularization classifier (last epoch):")
test_it(os.path.join(os.getcwd(), "checkpoint", "by_epoch", "gnreg_epoch_0200.pth"), norm_type="groupnorm")

Groupnorm classifier (best epoch):
	Accuracy on base dataset:	 0.8964757709251101
	Accuracy on affine dataset:	 0.737885462555066
	Accuracy on complex dataset:	 0.7389867841409692
Less dropout classifier (best epoch):
	Accuracy on base dataset:	 0.8887665198237885
	Accuracy on affine dataset:	 0.7070484581497798
	Accuracy on complex dataset:	 0.6398678414096917
More dropout classifier (best epoch):
	Accuracy on base dataset:	 0.9063876651982379
	Accuracy on affine dataset:	 0.7290748898678414
	Accuracy on complex dataset:	 0.7191629955947136
Groupnorm with regularization classifier (best epoch):
	Accuracy on base dataset:	 0.8667400881057269
	Accuracy on affine dataset:	 0.7125550660792952
	Accuracy on complex dataset:	 0.7004405286343612
Groupnorm classifier (last epoch):
	Accuracy on base dataset:	 0.9063876651982379
	Accuracy on affine dataset:	 0.7455947136563876
	Accuracy on complex dataset:	 0.724669603524229
Less dropout classifier (last epoch):
	Accuracy on base dataset:	 0.904